In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# case1plus

## 输出样例 task 以及 sample

In [ ]:
from samplers import get_data_sampler
from tasks import get_task_sampler
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import torch

In [ ]:
taskname = 'ode_ivp_case1plus'
dataname= 'ode_ivp_case1plus'

n_dims = 50
n_points = 41
bsize = 64
# n_points = 1
# bsize = 1
use_h = False
quadmethod='romberg'
# use_h = True

## xs 加载

In [ ]:
# 执行采样和计算
data_sampler = get_data_sampler(dataname,n_dims=n_dims)
xs = data_sampler.sample_xs(
    n_points,
    bsize,
    use_h = False,
)
torch.save(xs, f'test_{dataname}_xs.pt')
# h
data_sampler = get_data_sampler(dataname,n_dims=n_dims)
xs_h = data_sampler.sample_xs(
    n_points,
    bsize,
    use_h = True,
)
torch.save(xs_h, f'test_{dataname}_h_xs.pt')

In [ ]:
# xs = torch.load(f'test_{dataname}_xs.pt')
# # xs_h = torch.load(f'test_{dataname}_h_xs.pt')

## 预测

In [ ]:
print(f'{taskname}')
y_task = get_task_sampler(taskname,n_dims=n_dims,batch_size=bsize)
task_sampler = y_task()
ys = task_sampler.evaluate(xs,use_h = use_h)  # 获取模型预测结果

ode_ivp_case2vec


In [ ]:
print(f'{taskname}_h')
y_task = get_task_sampler(taskname,n_dims=n_dims,batch_size=bsize)
task_sampler = y_task()
ys = task_sampler.evaluate(xs,use_h = use_h)  # 获取模型预测结果

## 评估

In [ ]:
import numpy as np
import torch
from scipy.integrate import quad
import matplotlib.pyplot as plt

def analytical_solution(alpha_1, alpha_2, beta_1, beta_2, y_0, t):
    """
    计算原方程的解析解
    :param alpha_1: 方程中的参数 alpha_1
    :param alpha_2: 方程中的参数 alpha_2
    :param beta_1: 方程中的参数 beta_1
    :param beta_2: 方程中的参数 beta_2
    :param y_0: 初始条件 y(0)
    :param t: 时间点
    :return: 解析解在时间点 t 的值
    """
    def integrand(s):
        return beta_1 * np.exp((alpha_1 / 2) * s**2 + (alpha_2 + beta_2) * s)
    integral, _ = quad(integrand, 0, t,
                    epsabs=1e-12,
                    epsrel=1e-12,
                    limit = 100)
    return np.exp(-((alpha_1 / 2) * t**2 + alpha_2 * t)) * (integral + y_0)

# 假设 xs 是包含参数的张量，形状为 (batch_size, num_points, 7)
# 7 个参数分别为 [alpha_1, alpha_2, beta_1, beta_2, y_0, t_e, steps]
batch_idx = 2
point_idx = 0
alpha_1 = xs[batch_idx, point_idx, 0].item()
alpha_2 = xs[batch_idx, point_idx, 1].item()
beta_1 = xs[batch_idx, point_idx, 2].item()
beta_2 = xs[batch_idx, point_idx, 3].item()
y_0 = xs[batch_idx, point_idx, 4].item()
t_e = xs[batch_idx, point_idx, 5].item()
steps = int(xs[batch_idx, point_idx, 6].item())
params = [alpha_1, alpha_2, beta_1, beta_2, y_0, t_e, steps]
# 在参数提取后添加检查代码
print(f"当前参数检查：\n"
      f"alpha_1={alpha_1}\nalpha_2={alpha_2}\n"
      f"beta_1={beta_1}\nbeta_2={beta_2}\n"
      f"y0={y_0}\nt_e={t_e}\nsteps={steps}")

# 生成时间点
t_points = np.linspace(0, t_e, steps)

# 计算解析解
analytical_sols = np.array([analytical_solution(alpha_1, alpha_2, beta_1, beta_2, y_0, t) for t in t_points])
analytical_sols_2 = np.array([analytical_solution(alpha_1_, alpha_2_, beta_1_, beta_2_, y_0_, t) for t in t_points])
# 提取 ys 和 ys_rk 对应的数据
ys_data = ys[batch_idx, point_idx, :steps].cpu().detach().numpy()
ys_rk_data = ys_rk[batch_idx, point_idx, :steps].cpu().detach().numpy()

# 计算误差
mae_ys = np.mean(np.abs(ys_data - analytical_sols))
mae_ys_rk = np.mean(np.abs(ys_rk_data - analytical_sols))
mae_ana_ananew = np.mean(np.abs(analytical_sols - analytical_sols_2))
ys_err = np.abs(ys_data - analytical_sols)

print(f"ys的误差为：{ys_err}")
print(f"ys 与解析解的平均绝对误差 (MAE): {mae_ys}")
print(f"ys_rk 与解析解的平均绝对误差 (MAE): {mae_ys_rk}")
print(f"解析解的平均绝对误差 (MAE): {mae_ana_ananew}")

if mae_ys < mae_ys_rk:
    print("ys 的结果更接近解析解，可能 ys_rk 存在错误。")
elif mae_ys > mae_ys_rk:
    print("ys_rk 的结果更接近解析解，可能 ys 存在错误。")
else:
    print("ys 和 ys_rk 与解析解的误差相同。")

print(f"使用的向量化积分方式为：{quadmethod},\n此时的步长为:{h[batch_idx, point_idx].item()}\n精度核实为：O(h^{np.log(mae_ys)/np.log(h[batch_idx, point_idx].item())})")

# 绘图
plt.figure(figsize=(12, 8))
plt.plot(t_points, analytical_sols, label='Analytical Solution', color='black', linewidth=2)
plt.plot(t_points, analytical_sols_2, label='Analytical Solution_2', color='green', linewidth=2)
plt.plot(t_points, ys_data, label='ys', color='blue', linestyle='--')
plt.plot(t_points, ys_rk_data, label='ys_rk', color='red', linestyle='-.')
plt.title('Comparison of Analytical Solution, ys and ys_rk')
plt.xlabel('Time')
plt.ylabel('Solution Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

NameError: name 'xs' is not defined

# case 2

## 输出样例 task 以及 sample

In [25]:
from samplers import get_data_sampler
from tasks import get_task_sampler
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import torch

In [26]:
# taskname = 'ode_ivp_case2'
# dataname= 'ode_ivp_case2'

# n_points = 41
# n_dims = 50
# bsize = 64

In [27]:
# # 执行采样和计算
# data_sampler = get_data_sampler(dataname,n_dims=n_dims)
# xs = data_sampler.sample_xs(
#     n_points,
#     bsize,
#     n_dims_truncated=3  # 假设使用完整3维
# )
# y_task = get_task_sampler(taskname,n_dims=n_dims,batch_size=bsize)
# task_sampler = y_task()
# ys = task_sampler.evaluate(xs)  # 获取模型预测结果


In [28]:
# import matplotlib.pyplot as plt
# import torch

# # 可视化验证（新增部分）
# plt.figure(figsize=(10, 6))

# # 只取第一个 prompt 的前三个 point
# num_points_to_plot = 3
# t_ed = xs[1, 0, 5].item()  # 获取结束时间

# # 生成对应的 x 点
# x_points = []
# for i in range(num_points_to_plot):
#     steps = xs[1, i, 6].int().item()
#     x = torch.linspace(0, t_ed, steps)
#     x_points.append(x)

# # 定义不同的线条样式和颜色
# line_styles = ['-', '--', '-.']
# colors = ['r', 'g', 'b']

# # 绘制第一个 prompt 的前三个 point 的解
# for i in range(num_points_to_plot):
#     # 确保 Tensor 在 CPU 上并且不要求梯度计算
#     x_np = x_points[i].cpu().detach().numpy()
#     y_np = ys[1, i, :xs[1, i, 6].int().item()].cpu().detach().numpy()
    
#     # 确保每个绘图元素都有一个唯一的标签
#     plt.plot(x_np, y_np, 
#              linestyle=line_styles[i % len(line_styles)], 
#              color=colors[i % len(colors)], 
#              label=f'Solution {i+1}', 
#              alpha=0.7,  # 调整透明度
#              linewidth=1.5)  # 调整线条宽度

# plt.title('ODE IVP Solution Verification (First prompt, first 3 points)')
# plt.xlabel('Time steps')
# plt.ylabel('Value')
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
# taskname = 'ode_ivp_case1plus'
# dataname= 'ode_ivp_case1plus'
taskname = 'ode_ivp_case2vec'
dataname= 'ode_ivp_case2'

n_dims = 50
n_points = 41
bsize = 64
# n_points = 1
# bsize = 1
use_h = False
quadmethod='romberg'
# use_h = True

## xs 加载

In [30]:
# # 执行采样和计算
# data_sampler = get_data_sampler(dataname,n_dims=n_dims)
# xs = data_sampler.sample_xs(
#     n_points,
#     bsize,
#     use_h = use_h
# )
# torch.save(xs, f'test_{dataname}_xs.pt')

In [31]:
xs = torch.load(f'test_{dataname}_xs.pt')

In [32]:
print(xs[0,0,4].item())

0.11888301372528076


In [33]:
print(xs.dtype)

torch.float32


## 预测

In [34]:
h = (xs[..., 5] / xs[..., 6])
print(h)

tensor([[0.0330, 0.0265, 0.0381,  ..., 0.0813, 0.0871, 0.0871],
        [0.0515, 0.1504, 0.0416,  ..., 0.0698, 0.0752, 0.1630],
        [0.0892, 0.0474, 0.0758,  ..., 0.0345, 0.0337, 0.0689],
        ...,
        [0.0329, 0.0420, 0.0329,  ..., 0.0360, 0.0658, 0.0721],
        [0.0793, 0.0608, 0.0912,  ..., 0.0912, 0.2606, 0.0570],
        [0.1535, 0.0570, 0.1814,  ..., 0.0499, 0.2495, 0.2851]],
       device='cuda:0')


In [35]:
xs[0,0,6] = torch.tensor(50)

In [36]:
# y_task = get_task_sampler(taskname,n_dims=n_dims,batch_size=bsize)
# task_sampler = y_task()
# ys = task_sampler.evaluate(xs,use_h = use_h,quadmethod=quadmethod)  # 获取模型预测结果

In [37]:
print(taskname)
y_task = get_task_sampler(taskname,n_dims=n_dims,batch_size=bsize)
task_sampler = y_task()
ys = task_sampler.evaluate(xs,use_h = use_h)  # 获取模型预测结果

ode_ivp_case2vec


In [38]:
print(xs[0,0,4].item())

0.11888301372528076


In [39]:
y_task_truth = get_task_sampler('ode_ivp_case2',n_dims=n_dims,batch_size=bsize)
task_sampler_truth = y_task_truth()
ys_truth = task_sampler_truth.evaluate(xs,use_h = use_h)

In [40]:
print(xs[0,0,4].item())

0.11888301372528076


In [41]:
y_task_rk = get_task_sampler('ode_ivp_case2RK',n_dims=n_dims,batch_size=bsize)
task_sampler_rk = y_task_rk()
ys_rk = task_sampler_rk.evaluate(xs,use_h = use_h)

In [42]:
print(xs[0,0,4].item())

0.11888301372528076


In [43]:
print(torch.mean((ys-ys_truth)**2))

tensor(2.3184e-07, device='cuda:0')


In [44]:
print(torch.max((ys-ys_rk)))

tensor(0.0476, device='cuda:0')


In [45]:
print(torch.mean((ys-ys_rk)**2))

tensor(1.3102e-05, device='cuda:0')


In [46]:
print(torch.mean((ys_rk-ys_truth)**2))

tensor(1.2877e-05, device='cuda:0')


## 评估

In [4]:
import numpy as np
import torch
from scipy.integrate import quad
import matplotlib.pyplot as plt

def analytical_solution(alpha_1, alpha_2, beta_1, beta_2, y_0, t):
    """
    计算原方程的解析解
    :param alpha_1: 方程中的参数 alpha_1
    :param alpha_2: 方程中的参数 alpha_2
    :param beta_1: 方程中的参数 beta_1
    :param beta_2: 方程中的参数 beta_2
    :param y_0: 初始条件 y(0)
    :param t: 时间点
    :return: 解析解在时间点 t 的值
    """
    def integrand(s):
        return beta_1 * np.exp((alpha_1 / 2) * s**2 + (alpha_2 + beta_2) * s)
    integral, _ = quad(integrand, 0, t,
                    epsabs=1e-12,
                    epsrel=1e-12,
                    limit = 100)
    return np.exp(-((alpha_1 / 2) * t**2 + alpha_2 * t)) * (integral + y_0)

# 假设 xs 是包含参数的张量，形状为 (batch_size, num_points, 7)
# 7 个参数分别为 [alpha_1, alpha_2, beta_1, beta_2, y_0, t_e, steps]
batch_idx = 2
point_idx = 0
alpha_1 = xs[batch_idx, point_idx, 0].item()
alpha_2 = xs[batch_idx, point_idx, 1].item()
beta_1 = xs[batch_idx, point_idx, 2].item()
beta_2 = xs[batch_idx, point_idx, 3].item()
y_0 = xs[batch_idx, point_idx, 4].item()
t_e = xs[batch_idx, point_idx, 5].item()
steps = int(xs[batch_idx, point_idx, 6].item())
params = [alpha_1, alpha_2, beta_1, beta_2, y_0, t_e, steps]
# 在参数提取后添加检查代码
print(f"当前参数检查：\n"
      f"alpha_1={alpha_1}\nalpha_2={alpha_2}\n"
      f"beta_1={beta_1}\nbeta_2={beta_2}\n"
      f"y0={y_0}\nt_e={t_e}\nsteps={steps}")

# 生成时间点
t_points = np.linspace(0, t_e, steps)

# 计算解析解
analytical_sols = np.array([analytical_solution(alpha_1, alpha_2, beta_1, beta_2, y_0, t) for t in t_points])
analytical_sols_2 = np.array([analytical_solution(alpha_1_, alpha_2_, beta_1_, beta_2_, y_0_, t) for t in t_points])
# 提取 ys 和 ys_rk 对应的数据
ys_data = ys[batch_idx, point_idx, :steps].cpu().detach().numpy()
ys_rk_data = ys_rk[batch_idx, point_idx, :steps].cpu().detach().numpy()

# 计算误差
mae_ys = np.mean(np.abs(ys_data - analytical_sols))
mae_ys_rk = np.mean(np.abs(ys_rk_data - analytical_sols))
mae_ana_ananew = np.mean(np.abs(analytical_sols - analytical_sols_2))
ys_err = np.abs(ys_data - analytical_sols)

print(f"ys的误差为：{ys_err}")
print(f"ys 与解析解的平均绝对误差 (MAE): {mae_ys}")
print(f"ys_rk 与解析解的平均绝对误差 (MAE): {mae_ys_rk}")
print(f"解析解的平均绝对误差 (MAE): {mae_ana_ananew}")

if mae_ys < mae_ys_rk:
    print("ys 的结果更接近解析解，可能 ys_rk 存在错误。")
elif mae_ys > mae_ys_rk:
    print("ys_rk 的结果更接近解析解，可能 ys 存在错误。")
else:
    print("ys 和 ys_rk 与解析解的误差相同。")

print(f"使用的向量化积分方式为：{quadmethod},\n此时的步长为:{h[batch_idx, point_idx].item()}\n精度核实为：O(h^{np.log(mae_ys)/np.log(h[batch_idx, point_idx].item())})")

# 绘图
plt.figure(figsize=(12, 8))
plt.plot(t_points, analytical_sols, label='Analytical Solution', color='black', linewidth=2)
plt.plot(t_points, analytical_sols_2, label='Analytical Solution_2', color='green', linewidth=2)
plt.plot(t_points, ys_data, label='ys', color='blue', linestyle='--')
plt.plot(t_points, ys_rk_data, label='ys_rk', color='red', linestyle='-.')
plt.title('Comparison of Analytical Solution, ys and ys_rk')
plt.xlabel('Time')
plt.ylabel('Solution Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

NameError: name 'xs' is not defined

## 验证代码问题

In [ ]:
# print(xs[0,0,4].item())

In [ ]:
# paras = torch.load("paras.pt")
# taskpara_tensor  = torch.load("params_tensor.pt")
# taskpara = np.load("params_debug.npy")
# taskpara = taskpara[0,0]
# taskpara_tensor = taskpara_tensor[0,0]

In [ ]:
# print(paras[4])
# print(taskpara[4])
# print(taskpara_tensor[4].item())

In [ ]:
# print(xs[0,0,:7])

In [ ]:
# print(xs.dtype)
# print(taskpara_tensor.dtype)
# print(type(paras[0]))

In [ ]:
for i in range(100):
    data_sampler = get_data_sampler(dataname,n_dims=n_dims)
    xs = data_sampler.sample_xs(
        n_points,
        bsize,
        use_h = use_h
    )
    y_task = get_task_sampler(taskname,n_dims=n_dims,batch_size=bsize)
    task_sampler = y_task()
    ys = task_sampler.evaluate(xs,use_h = use_h)  # 获取模型预测结果

In [ ]:
# 在计算解析解之后添加验证代码
t_target = t_e  # 测试终点
direct_integral, _ = quad(
    lambda s: beta_1 * np.exp((alpha_1/2)*s**2 + (alpha_2 + beta_2)*s),
    0, t_target,
    epsabs=1e-15,
    epsrel=1e-15,
    limit=500
)

cumulative_integral = sum(
    quad(
        lambda s: beta_1 * np.exp((alpha_1/2)*s**2 + (alpha_2 + beta_2)*s),
        pre_t, curr_t,
        epsabs=1e-15,
        epsrel=1e-15,
        limit=500
    )[0]
    for pre_t, curr_t in zip(t_points[:-1], t_points[1:])
)

print(f"直接积分结果: {direct_integral:.15f}")
print(f"累积积分结果: {cumulative_integral:.15f}")
print(f"绝对差异: {abs(direct_integral - cumulative_integral):.2e}")

## loss 测试

In [ ]:
steptable = task_sampler.get_training_steptable(xs,use_h = use_h).cuda()
# 增强损失 每个数据点计算前几个的损失
loss_func = task_sampler.get_enhanced_training_metric(steptable)  # 损失metric
point_wise_loss_func = task_sampler.get_enhanced_metric(steptable)  # loss_func

In [ ]:
min_val = steptable.min().item()  # 获取全局最小值
max_val = steptable.max().item()
print(min_val, max_val)